In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/commercial_area.csv", dtype={"행정구역코드": "object"})

In [3]:
# 시군구 추출
area_code = {k: v for v, k in df[df["구분"] == "gu"][["행정구역", "행정구역코드"]].values}

# 행정동 추출
df = df[df["구분"] == "dong"].reset_index(drop=True)
df["시군구"] = df["행정구역코드"].apply(lambda x: area_code[x[:5]])

df.drop(["행정구역코드", "구분"], axis=1, inplace=True)
df = df[[df.columns[-1]] + list(df.columns[:-1])]

In [4]:
result = df[["시군구", "행정구역"]].copy()

In [5]:
for col in df.columns:
    if "2022" not in col:
        continue

    # 점포수
    if "점포수" in col:
        # 점포수 개수
        result[col] = df[col]
        # 상위 -%
        result[col + " 등급"] = round((df[col].rank(method="max", ascending=False) / df.shape[0]) * 100, 1)
        # 작년 대비 증가 개수
        result[" ".join(col.split(" ")[1:] + ["변화율"])] = round(df[col] - df[col.replace("2022", "2021")], 2)
    # 신생기업 생존율
    elif "신생기업" in col:
        result[col] = df[col].replace("-", "")
    # 평균영업기간
    elif "10년기준" in col:
        result[col] = df[col].round(1)
        # 상위 -%
        result[col + " 등급"] = round((df[col].rank(method="max", ascending=False) / df.shape[0]) * 100, 1)
        # 작년 대비 증가 비율
        result[" ".join(col.split(" ")[1:] + ["변화율"])] = round((df[col] / df[col.replace("2022", "2021")] * 100) - 100, 1)
    # 개업수
    elif "개업수" in col or "폐업수" in col or "개업률" in col or "폐업률" in col:
        result[col] = df[col]
        # 상위 -%
        result[col + " 등급"] = round((df[col].rank(method="max", ascending=False) / df.shape[0]) * 100, 1)
    # 인구수
    elif "인구" in col:
        result[col] = df[col]
        # 상위 -%
        result[col + " 등급"] = round((df[col].rank(method="max", ascending=False) / df.shape[0]) * 100, 1)
    # 소득/가구수
    elif "소득분위" in col or "가구수" in col:
        result[col] = df[col]
        # 상위 -%
        result[col + " 등급"] = round((df[col].rank(method="max", ascending=False) / df.shape[0]) * 100, 1)
    # 임대시세
    elif "환산임대료" in col and "전체" in col:
        df[col] = df[col].replace(",", "").replace(" ", "").apply(lambda x: int(x) if x else 0)
        df[col.replace("2022", "2021")] = df[col.replace("2022", "2021")].replace(",", "").replace(" ", "").apply(lambda x: int(x) if x else 0)
        
        result[col] = df[col]
        # 상위 -%
        result[col + " 등급"] = round((df[col].rank(method="max", ascending=False) / df.shape[0]) * 100, 1)
        # 작년 대비 증가 비율
        result[" ".join(col.split(" ")[1:] + ["변화율"])] = round((df[col] / df[col.replace("2022", "2021")] * 100) - 100, 1)

In [6]:
# 점포수
# 외식업의 패스트푸드 전체 점포수는 몇개로 서울 행정동내에서 상위(하위) -%이며 작년 대비 증가(감소) 했다.
# 외식업의 패스트푸드 프렌차이점 개수는 몇개로 서울 행정동내에서 상위(하위) -%이며 작년 대비 증가(감소) 했다.
# 외식업의 패스트푸드 일반 점포수는 몇개로 서울 행정동내에서 상위(하위) -%이며 작년 대비 증가(감소) 했다.

# 신생기업 생존율
# 외식업의 패스트푸드 1년 생존율은 약 -%이다.
# 외식업의 패스트푸드 3년 생존율은 약 -%이다.
# 외식업의 패스트푸드 5년 생존율은 약 -%이다.

# 평균영업기간
# 외식업의 패스트푸드 최근 10년기준 평균 영업기간은 -년으로 행정동내에서 상위(하위) -%이다.
# 외식업의 패스트푸드 작년 대비 최근 10년기준 평균 영업기간은 -% 증가(감소)했다.

# 개/폐업수(률)
# 외식업의 패스트푸드 개업수는 몇개로 서울 행정동내에서 상위(하위) -%이다.
# 외식업의 패스트푸드 폐업수는 몇개로 서울 행정동내에서 상위(하위) -%이다.
# 외식업의 패스트푸드 개업율은 몇 %로 서울 행정동내에서 상위(하위) -%이다.
# 외식업의 패스트푸드 폐업율는 몇 %로 서울 행정동내에서 상위(하위) -%이다.

# 인구수
# 길단위 유동인구 수는 -명이며 서울 행정동내에서 상위(하위) -%이다.
# 주거인구 수는 -명이며 서울 행정동내에서 상위(하위) -%이다.
# 직장인구 수는 -명이며 서울 행정동내에서 상위(하위) -%이다.
# 작년 대비 길단위 유동인구 수 -% 증가(감소)했다.
# 작년 대비 주거인구 수는 -% 증가(감소)했다.
# 작년 대비 직장인구 수는 -% 증가(감소)했다.

# 소득/가구수
# 소득분위는 몇분위로 (a-b)사이 이며 서울 행정동내에서 상위(하위) -%이다.
# 가구수 -이며 서울 행정동내에서 상위(하위) -%이다.

# 임대시세
# 1평당 월 임대시세는 -이며 서울 행정동내 상위(하위) -%이다.
# 직년 대비 1평당 월 임대시세는 -% 상승(감소)했다.

In [8]:
result.to_csv("data/preprocessed_commercial_area.csv", index=False)